# Planet monthly mosaic data

* **Products used:**
[DE Africa Waterbodies](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html),
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a),
[Planet API]()


## Background

Norway's International Climate and Forests Initiative (NICFI) is an international development fund that helps save the world's tropical forests while improving the livelihoods of those who live off, in, and near forests. In September 2020, the Norwegian Ministry of Climate and Environment awarded an international contract to Kongsberg Satellite Services (KSAT), with partners Planet and Airbus, to provide comprehensive access to high-resolution satellite monitoring of the tropics. As part of this program, Planet partnered with NICFI to make high-resolution satellite imagery of the tropics available free of charge to users, advancing the NICFI purpose of reducing and reversing tropical forest loss, combating climate change, conserving biodiversity, and facilitating sustainable development. Additional information on the program can be found on the [NICFI program website](https://www.planet.com/nicfi). 

Digital Earth Africa has enabled users to view NICFI's latest monthly and biannual high-resolution (<5 m) mosaics through its platform (Maps and Sandbox). These visual mosaics provide optimized, true-color imagery, making them ideal for visual display and interpretation. For more information and access to additional visual and analytic products from Planet, visit the [NICFI program website](https://www.planet.com/nicfi)

**Important details:**

* Planet Monthly Mosaic
* Spatial resolution: 4.77m
* Bands: Red, Green, Blue (3-band)
* Temporal resolution: monthly
* Temporal range: 2020-09 &ndash; present
* Update frequency: Monthly
* Update latency: Max 5 days from previous month
* Api access: `https://api.digitalearth.africa/planet/tiles/basemaps/v1/planet-tiles/planet_medres_visual_year-month_mosaic/gmap/{z}/{x}/{y}.png`
---
* Archive Planet biannual Mosaic
* Spatial resolution: 4.77m
* Bands: Red, Green, Blue (3-band)
* Temporal resolution: 6 months
* Temporal range: 2015-12 &ndash; 2020-08
* [Visualise on DE Africa Maps](https://maps.digitalearth.africa/#share=s-3nQm2WLyXLl1LahsxI1JJ5bgWFO)


For a detailed description visit [NICFI_UserGuidesFAQ.pdf](https://assets.planet.com/docs/NICFI_UserGuidesFAQ.pdf).

## Description
This notebook demonstrates how to use the Planet monthly mosaic data in conjunction with other DE Africa services. 

Topics covered include:

1. Define area of interest
2. Plotting of waterbody, Normalized Difference Vegetation Index (NDVI) and Build-Up Index (BUI) with planet monthly mosaic as basemap

***

## Getting started



### Load packages
Import Python packages that are used for the analysis.

- Uncomment the below cell if `localtileserver` throws up an error and rerun the cell
- Once installed comment it out.

In [1]:
#pip install localtileserver

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

In [2]:
import geopandas as gpd
from datacube.utils.geometry import Geometry
from deafrica_tools.plotting import display_map
from deafrica_tools.areaofinterest import define_area

from deafrica_tools.app.planetbasemap import loadplanet

### Analysis parameters

The following cell sets the parameters, which define the area of interest and the length of time to conduct the analysis over.
The parameters are

* `lat`: The central latitude to analyse (e.g. `10.338`).
* `lon`: The central longitude to analyse (e.g. `-1.055`).
* `buffer`: The number of square degrees to load around the central latitude and longitude. 

#### Select location
To define the area of interest, there are two methods available:

1. By specifying the latitude, longitude, and buffer. This method requires you to input the central latitude, central longitude, and the buffer value in square degrees around the center point you want to analyze. For example, `lat = 10.338`, `lon = -1.055`, and `buffer = 0.1` will select an area with a radius of 0.1 square degrees around the point with coordinates (10.338, -1.055).

2. By uploading a polygon as a `GeoJSON or Esri Shapefile`. If you choose this option, you will need to upload the geojson or ESRI shapefile into the Sandbox using Upload Files button <img align="top" src="../Supplementary_data/upload_files_icon.png"> in the top left corner of the Jupyter Notebook interface. ESRI shapefiles must be uploaded with all the related files `(.cpg, .dbf, .shp, .shx)`. Once uploaded, you can use the shapefile or geojson to define the area of interest. Remember to update the code to call the file you have uploaded.

To use one of these methods, you can uncomment the relevant line of code and comment out the other one. To comment out a line, add the `"#"` symbol before the code you want to comment out. By default, the first option which defines the location using latitude, longitude, and buffer is being used.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.
The example covers the Bui Dam in Ashanti Region, Ghana.


In [3]:
# Method 1: Specify the latitude, longitude, and buffer
aoi = define_area(lat=6.74248, lon=-1.69340, buffer=0.05)

# Method 2: Use a polygon as a GeoJSON or Esri Shapefile. 
# aoi = define_area(vector_path='aoi.shp')

geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)

# Get the latitude and longitude range of the geopolygon
lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])
lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])

## View the selected location
The next cell will display the selected area on an interactive map.
Feel free to zoom in and out to get a better understanding of the area you'll be analysing.
Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [4]:
display_map(x=lon_range, y=lat_range, zoom_bias=1)

## Plotting of Result

The cell below plots the map with `ipyleaflet` functions, which load `opensteetmap` and the `Latest Planet monthly mosaic` as the basemap. It also loads the

- `Built up area `       : generated using Build-Up Index (BUI)
- `Vegetation    `       : generated using Normalized Difference Vegetation Index (NDVI) 
- `Water body    `       : maximum extent mapped in Waterbodies service

The NDVI and BUI were computed from Sentinel-2 Annual GeoMad. More information on the [GeoMAD](GeoMAD.ipynb).

The loadplanet function takes two parameters 
- `lon_range   ` : longitude buffer for the study area
- `lat_range   ` : latitude  buffer for the study area

The next cell will display the selected area on an interactive map.
Feel free to zoom in and out to get a better understanding of the area you'll be analysing.

In [5]:
#NDVI threshold can be adjusted below; the NDVI value range is between 0 and 1, default value used is 0.6
threshold_nvdi = 0.6
#BUI threshold can be adjsuted below, BUI value range is between -1 and 1, default value used is 0
threshold_bui = 0
loadplanet(lon_range, lat_range, threshold_nvdi, threshold_bui)

Map(center=[6.7424885, -1.6934010000000002], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [6]:
import datacube
print(datacube.__version__)

1.8.15


**Last Tested:**

In [7]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2024-07-02'